# Create Naive Bayes Model 

Given the cosine similarity fatures and the embeddings, create model and evalute how well the model does at predicting he appropriate label 

In [62]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [77]:
pd.set_option('display.max_rows', 100)

In [70]:
# ================================
# STEP 1: Load Dataset 
# ================================
import sys

# set this on the path so that we can reference the commong data locations
sys.path.append("../scripts/")

from data_collection import authenticate_google_drive, grab_google_drive_folder_data

drive = authenticate_google_drive('0_data_collection/credentials/google_drive_client_secret.json')
file_path = "labelled_consine_similarity.csv"
data  = grab_google_drive_folder_data(drive=drive,credential_file="0_data_collection/credentials/google_drive_folder_id.json",filename=file_path)



Successfully loaded 'labelled_consine_similarity.csv' into a DataFrame!


In [71]:
data.head() 

,Unnamed: 0,submission_id,subredit_topic,search_query,combine_text,url,label,reviewer,title,text,...,multi-qa-mpnet-base-dot-v1_positive,multi-qa-mpnet-base-dot-v1_negative,thenlper/gte-large_positive,thenlper/gte-large_negative,BAAI/bge-m3_positive,BAAI/bge-m3_negative,intfloat/multilingual-e5-small_positive,intfloat/multilingual-e5-small_negative,positive_query,negative_query
0,0,1hcgjtw,personalfinance,Rocket Mortgage,Banker is trying to talk me into a cashout ref...,https://www.reddit.com/r/FirstTimeHomeBuyer/1h...,Neutral,aserban,Banker is trying to talk me into a cashout ref...,"A little background, trying to borrow \~100k f...",...,0.273178,0.257725,0.708095,0.682507,0.430416,0.397636,0.775435,0.786102,I've had a fantastic experience with this comp...,"Terrible experience with this company, never u..."
1,1,1irwq2g,tax,Turbo Tax,Should we use turbo tax or get a tax professio...,https://www.reddit.com/r/FirstTimeHomeBuyer/1i...,Neutral,aserban,Should we use turbo tax or get a tax professio...,I have used TT for at least the last 7 years. ...,...,0.337967,0.294918,0.715634,0.686359,0.448315,0.393052,0.776792,0.802141,I've had a fantastic experience with this comp...,"Terrible experience with this company, never u..."
2,2,1eniv3b,personalfinance,Wells Fargo,"Roth IRA w/Wells Fargo, rollover IRA w/Fidelit...",https://www.reddit.com/r/FirstTimeHomeBuyer/1e...,Neutral,aserban,"Roth IRA w/Wells Fargo, rollover IRA w/Fidelit...","I have around $10,000 in a WellsFargo Roth IRA...",...,0.230517,0.194605,0.710653,0.686760,0.428669,0.392935,0.797070,0.804273,I've had a fantastic experience with this comp...,"Terrible experience with this company, never u..."
3,3,1ix1aur,IRS,Turbo Tax,Turbo Tax couldn’t deliver on the 5 day early ...,https://www.reddit.com/r/FirstTimeHomeBuyer/1i...,Negative,aserban,Turbo Tax couldn’t deliver on the 5 day early ...,Has this happened to anyone else? Has anyone g...,...,0.410005,0.483772,0.736017,0.739138,0.502871,0.503127,0.779282,0.815123,I've had a fantastic experience with this comp...,"Terrible experience with this company, never u..."
4,4,1dmahi0,RealEstate,Zillow,Propane or NG on Zillow?. I am looking to buy ...,https://www.reddit.com/r/FirstTimeHomeBuyer/1d...,Neutral,aserban,Propane or NG on Zillow?,I am looking to buy in a few years. I was hopi...,...,0.205350,0.175084,0.702612,0.690053,0.343543,0.378241,0.766830,0.787916,I've had a fantastic experience with this comp...,"Terrible experience with this company, never u..."


In [78]:
# Handle null value cases 
# Happens when text is null but title is not 
# data[data['combine_text'].isna() ].head() 

data['combine_text'] = data['combine_text'].combine_first(data['title'])


,Unnamed: 0,submission_id,subredit_topic,search_query,combine_text,url,label,reviewer,title,text,...,multi-qa-mpnet-base-dot-v1_positive,multi-qa-mpnet-base-dot-v1_negative,thenlper/gte-large_positive,thenlper/gte-large_negative,BAAI/bge-m3_positive,BAAI/bge-m3_negative,intfloat/multilingual-e5-small_positive,intfloat/multilingual-e5-small_negative,positive_query,negative_query
20,20,1dpzw2y,realtors,Zillow,Greetings Fellow Realtors! Have any of you uti...,https://www.reddit.com/r/FirstTimeHomeBuyer/1d...,Neutral,aserban,Greetings Fellow Realtors! Have any of you uti...,NaN,...,0.522622,0.466992,0.722843,0.701871,0.474688,0.437406,0.824518,0.820082,I've had a fantastic experience with this comp...,"Terrible experience with this company, never u..."


In [48]:
data.shape[0]

43

In [57]:
# ================================
# 🔥 STEP 2: Preprocess the Data
# ================================

# Convert labels to binary values (ham=0, spam=1)
data['label_model'] = data['label'].map({'Neutral': 0, 'Negative': 1, 'Positive': 2})

# Vectorize the text using CountVectorizer (bag-of-words)
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(data['combine_text'])
y = data['label_model']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)



✅ Accuracy: 0.7272727272727273

✅ Confusion Matrix:
 [[7 1 0]
 [1 1 1]
 [0 0 0]]

✅ Classification Report:
               precision    recall  f1-score   support

     Neutral       0.88      0.88      0.88         8
    Negative       0.50      0.33      0.40         3
    Positive       0.00      0.00      0.00         0

    accuracy                           0.73        11
   macro avg       0.46      0.40      0.42        11
weighted avg       0.77      0.73      0.75        11



/Users/andreeaserban/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/andreeaserban/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/andreeaserban/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

# ================================
# 🔥 STEP 3: Train Naive Bayes Model
# ================================

# Use MultinomialNB since it's ideal for text classification
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Make predictions
y_pred = nb_model.predict(X_test)



In [ ]:

# ================================
# 🔥 STEP 4: Evaluate the Model
# ================================

# Accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=["Neutral", "Negative", "Positive"])

print("\n✅ Accuracy:", accuracy)
print("\n✅ Confusion Matrix:\n", conf_matrix)
print("\n✅ Classification Report:\n", report)


In [60]:


# ================================
# 🔥 STEP 5: Example Predictions
# ================================

# Test some sample messages
sample_messages = [
    "I love Rocket! ",
    "I hate Rocket Mortgage",
    "Honest review of Rocket Mortgage: It is great"
]

# Transform the sample messages
sample_vectorized = vectorizer.transform(sample_messages)

# Make predictions
predictions = nb_model.predict(sample_vectorized)

# Display results
for i, msg in enumerate(sample_messages):
    if predictions[i] == 0: 
        label = 'Neutral'
    elif predictions[i] == 1: 
        label='Negative'
    elif predictions[i] == 2: 
        label='Positive'
    else:
        print("Error")
    print(f"\n📩 Message: {msg}\n🔍 Prediction: {label}")



📩 Message: I love Rocket! 
🔍 Prediction: Neutral

📩 Message: I hate Rocket Mortgage
🔍 Prediction: Neutral

📩 Message: Honest review of Rocket Mortgage: It is great
🔍 Prediction: Neutral
